In [1]:
import pandas as pd
import os
from styleframe import StyleFrame

Este notebook transforma os dados do formato original (`./analise-amostragem-imdb-SRS/data/`) para o formato suportado neste trabalho (`./data_formated/`)

In [2]:
approachs = os.listdir("data")
approachs

['approach2.xlsx',
 'approach1.xlsx',
 'random.xlsx',
 'approach4.xlsx',
 'approach3.xlsx',
 'approach5.xlsx']

Marca tudo como VP ou VN

In [54]:
for approach in approachs:
    df = pd.read_excel("data/" + approach, sheet_name=None)
    for t in df.keys():
        if len(df[t].columns) > 4:
            df[t] = df[t].rename(columns={'Unnamed: 4': "subclassification"})
        else:
            df[t]["subclassification"] = ""
        df[t].insert(4, "classification", df[t]["succeed"].map(lambda x: "VP" if x==0 else "VN"))
        df[t]["obs"] = ""

    with pd.ExcelWriter("data_formated/" + approach) as writer:
        for i, _df in df.items():
            _df.to_excel(writer, sheet_name=i, index=False)


In [4]:
df = pd.read_excel("data/approach1.xlsx", sheet_name=None)

In [8]:
df["template1"] = df["template1"].rename(columns={'Unnamed: 4': "subclassification"})

In [13]:
df["template1"]

,text,label,pred,succeed,subclassification
0,Things do n't exactly expected out as expected .,0,1,0,NaN
1,Things do n't exactly keep out as keep .,0,1,0,LE
2,Things do n't exactly wears out as wears .,0,0,1,LE


In [23]:
import numpy as np

Mantem os rótulos de Adson

In [61]:
"""
- verde escuro: verdadeiro positivo
- verde claro: falso positivo
- vermelho escuro: verdadeiro negativo
- vermelho claro: falso negativo
"""

def is_VP(cell):
    return cell.style.bg_color.upper() == "FF93C47D" # verde escuro
def is_VN(cell):
    return cell.style.bg_color.upper() == "FFE06666"  # vermelho escuro
def is_FP(cell):
    return cell.style.bg_color.upper() == "FFD9EAD3" # verde claro
def is_FN(cell):
    return cell.style.bg_color.upper() == "FFF4CCCC"  # vermelho claro

def create_classification(cell):
    classification = None
    if is_FN(cell):
        classification="FN"
    elif is_FP(cell):
        classification="FP"
    elif is_VP(cell):
        classification="VP"    
    elif is_VN(cell):
        classification="VN"

    assert bool(classification), cell.style
    
    return classification

In [62]:
for approach in ["approach5.xlsx"]:
    filename_to_read = "data/" + approach
    df = pd.read_excel(filename_to_read, sheet_name=None)
    for t in df.keys():
        sf = StyleFrame.read_excel(filename_to_read, read_style=True, sheet_name=t)
        print(t)

        
        if len(sf.columns) > 4:
            sf = sf.rename(columns={'Unnamed: 4': "subclassification"})
        else:
            sf["subclassification"] = [""]*len(sf)
        ## aqui entra a mágica das cores
        sf.data_df.insert(4, "classification", sf["text"].map(create_classification))
        sf["obs"] = [""]*len(sf)

        df[t] = sf.data_df
    with pd.ExcelWriter("data_formated_label_adson/" + approach) as writer:
        for i, _df in df.items():
            _df.to_excel(writer, sheet_name=i, index=False)


template1
template2
template5
template3
template4
template6
template7
template8
template9
template10
template11
template12
template13
template14
template15
template16
template17
template18
template19
template20
template21
template22
template23
template24
template25
template26


In [47]:
sf.data_df.insert(4, "classification", sf["text"].map(create_classification))

In [52]:
sf["obs"] = [""]*len(sf)
